In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from haversine import haversine

sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

import platform
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)

import scipy as sp
import statsmodels.api as sm # statsmodel 기본 import
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms 
import sklearn as sk
from patsy import dmatrix

import warnings
warnings.filterwarnings("ignore")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### train데이터 변형

In [2]:
train = pd.read_csv("train.csv")
train.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [3]:
train.tail(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373
1458643,id1209952,1,2016-04-05 14:44:25,2016-04-05 14:47:43,1,-73.979538,40.781750,-73.972809,40.790585,N,198


##### 데이터 전처리
- date 분리 : date / month / weekday / time 추가
- distance 추가

##### date 분리 : date / month / weekday / time 추가

In [4]:
pickup_datetime_dt = pd.to_datetime(train["pickup_datetime"])
dropoff_datetime_dt = pd.to_datetime(train["dropoff_datetime"])

train["pickup_datetime"] = pickup_datetime_dt   #datetime64 형식으로 바꿔줌
train["dropoff_datetime"] = dropoff_datetime_dt #datetime64 형식으로 바꿔줌

train["pickup_date"] = train["pickup_datetime"].dt.date              #date only
train["dropoff_date"] = train["dropoff_datetime"].dt.date            #date only

train["pickup_month"] = train["pickup_datetime"].dt.month            #month only
train["dropoff_month"] = train["dropoff_datetime"].dt.month          #month only

train["pickup_weekday"] = train["pickup_datetime"].dt.weekday        #weekday only (요일)
train["dropoff_weekday"] = train["dropoff_datetime"].dt.weekday      #weekday only (요일)

train["pickup_hour"] = train["pickup_datetime"].dt.hour              #hour only
train["dropoff_hour"] = train["dropoff_datetime"].dt.hour            #hour only

##### distance 추가

In [5]:
### 승하차 위치 데이터 tuple로 묶기 - pickup_loc / dropoff_loc로 리턴

In [6]:
pickup_lat = tuple(train["pickup_latitude"])
pickup_lng = tuple(train["pickup_longitude"])
dropoff_lat = tuple(train["dropoff_latitude"])
dropoff_lng = tuple(train["dropoff_longitude"])

In [7]:
pickup_loc = tuple(zip(pickup_lat, pickup_lng))
dropoff_loc = tuple(zip(dropoff_lat, dropoff_lng))

In [8]:
len(pickup_loc), len(dropoff_loc)

(1458644, 1458644)

In [9]:
### pickup_loc & dropoff_loc 으로 직선거리 구하기

In [10]:
distance = []
for i in range(len(pickup_loc)):
    distance.append(haversine(pickup_loc[i], dropoff_loc[i]))

In [11]:
train["distance"] = distance

In [12]:
len(distance)

1458644

In [13]:
train.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,pickup_month,dropoff_month,pickup_weekday,dropoff_weekday,pickup_hour,dropoff_hour,distance
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02,3.516818e+00,3.517178e+00,3.050375e+00,3.054422e+00,1.360648e+01,1.359890e+01,3.440864e+00
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03,1.681038e+00,1.681217e+00,1.954039e+00,1.956599e+00,6.399693e+00,6.483682e+00,4.296538e+00
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,9.000000e+00,9.000000e+00,1.231837e+00
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02,4.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,1.400000e+01,1.400000e+01,2.093717e+00
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,1.900000e+01,1.900000e+01,3.875337e+00
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06,6.000000e+00,7.000000e+00,6.000000e+00,6.000000e+00,2.300000e+01,2.300000e+01,1.240909e+03


### test 데이터 변형

In [14]:
test = pd.read_csv("test.csv")
test.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000
mean,1.534884,1.661765,-73.973614,40.750927,-73.973458,40.751816
std,0.498782,1.311293,0.073389,0.029848,0.072565,0.035824
min,1.000000,0.000000,-121.933128,37.389587,-121.933327,36.601322
25%,1.000000,1.000000,-73.991852,40.737392,-73.991318,40.736000
50%,2.000000,1.000000,-73.981743,40.754093,-73.979774,40.754543
75%,2.000000,2.000000,-73.967400,40.768394,-73.963013,40.769852
max,2.000000,9.000000,-69.248917,42.814938,-67.496796,48.857597


In [15]:
test.tail(5)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N
625133,id0621643,2,2016-01-01 00:00:22,2,-73.981850,40.716881,-73.969330,40.769379,N


##### 데이터 전처리
- date 분리 : date / month / weekday / time 추가
- distance 추가

##### date 분리 : date / month / weekday / time 추가

In [16]:
pickup_datetime_dt = pd.to_datetime(test["pickup_datetime"])

test["pickup_datetime"] = pickup_datetime_dt   #datetime64 형식으로 바꿔줌

test["pickup_date"] = test["pickup_datetime"].dt.date              #date only

test["pickup_weekday"] = test["pickup_datetime"].dt.weekday        #weekday only (요일)

test["pickup_hour"] = test["pickup_datetime"].dt.hour              #hour only

##### distance 추가

In [17]:
### 승하차 위치 데이터 tuple로 묶기 - pickup_loc / dropoff_loc로 리턴

In [18]:
pickup_lat = tuple(test["pickup_latitude"])
pickup_lng = tuple(test["pickup_longitude"])
dropoff_lat = tuple(test["dropoff_latitude"])
dropoff_lng = tuple(test["dropoff_longitude"])

In [19]:
pickup_loc = tuple(zip(pickup_lat, pickup_lng))
dropoff_loc = tuple(zip(dropoff_lat, dropoff_lng))

In [20]:
len(pickup_loc), len(dropoff_loc)

(625134, 625134)

In [21]:
### pickup_loc & dropoff_loc 으로 직선거리 구하기

In [22]:
distance = []
for i in range(len(pickup_loc)):
    distance.append(haversine(pickup_loc[i], dropoff_loc[i]))

In [23]:
test["distance"] = distance

In [24]:
len(distance)

625134

#### trip_duration ~ log1p(distance) + abs(np.sin(pickup_hour)) + 0 
- 현재 모델 : r-square 92 / cond 3.82 / F-statistic 8.349e+06
- passenger_count 추가시 : r-square 0.931 / cond 6.32 / F-statistic 6.571e+06

In [25]:
model = sm.OLS.from_formula("np.log1p(trip_duration) ~ np.log1p(distance) + abs(np.sin(pickup_hour)) + 0" , train)
result = model.fit()
print(result.summary())

                               OLS Regression Results                              
Dep. Variable:     np.log1p(trip_duration)   R-squared:                       0.920
Model:                                 OLS   Adj. R-squared:                  0.920
Method:                      Least Squares   F-statistic:                 8.349e+06
Date:                     Sat, 10 Mar 2018   Prob (F-statistic):               0.00
Time:                             17:39:29   Log-Likelihood:            -2.9646e+06
No. Observations:                  1458644   AIC:                         5.929e+06
Df Residuals:                      1458642   BIC:                         5.929e+06
Df Model:                                2                                         
Covariance Type:                 nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------

In [26]:
test_new = pd.DataFrame(columns=["distance", "pickup_hour"])
test_new["distance"] = test["distance"]
test_new["pickup_hour"] = test["pickup_hour"]
test_new

,distance,pickup_hour
0,2.746426,23
1,2.759239,23
2,1.306155,23
3,5.269088,23
4,0.960842,23
5,4.186272,23
6,3.512614,23
7,2.980953,23
8,18.852171,23
9,1.820015,23


In [27]:
y_new = result.predict(test_new)

In [28]:
# test에 따른 y_new산출
y_new

0          6.889687
1          6.899558
2          5.486946
3          8.378023
4          5.018014
5          7.829856
6          7.427616
7          7.065222
8         11.710353
9          6.068488
10         8.129739
11         4.751767
12         5.748380
13         6.782906
14         8.454043
15         8.156751
16         7.476723
17         5.994412
18         4.589043
19         8.509692
20         4.674181
21         6.232024
22         5.947154
23         8.125070
24         8.364787
25         9.269146
26         6.698250
27         5.216723
28        10.064900
29         9.424551
            ...    
625104     2.634370
625105     2.474592
625106     3.476176
625107     2.762977
625108     6.298347
625109     4.202538
625110     1.402460
625111     4.713454
625112     2.565627
625113     3.187639
625114     3.066382
625115     8.178663
625116     3.589638
625117     6.700592
625118     5.558032
625119     2.660658
625120     2.672002
625121     1.787794
625122     2.296916


In [29]:
y_new.sum()

3668949.5807281001

In [67]:
submission = pd.DataFrame(y_new, columns=["trip_duration"])

In [63]:
# test에서 id추출
id = test["id"]
id

0         id3004672
1         id3505355
2         id1217141
3         id2150126
4         id1598245
5         id0668992
6         id1765014
7         id0898117
8         id3905224
9         id1543102
10        id3024712
11        id3665810
12        id1836461
13        id3457080
14        id3376065
15        id3008739
16        id0902216
17        id3564824
18        id0820280
19        id0775088
20        id1468488
21        id2657479
22        id1262719
23        id1345524
24        id2911638
25        id2849512
26        id0236829
27        id2905906
28        id3737939
29        id0766179
            ...    
625104    id0120169
625105    id0386349
625106    id1962532
625107    id0335207
625108    id0273508
625109    id2936770
625110    id0044526
625111    id3605431
625112    id2681896
625113    id3308448
625114    id2108525
625115    id3952220
625116    id2771348
625117    id3065313
625118    id2332834
625119    id3495407
625120    id3811106
625121    id2693698
625122    id2884571


In [79]:
# 제출양식 작성
final = pd.concat([id, submission], axis=1)
final

,id,trip_duration
0,id3004672,6.889687
1,id3505355,6.899558
2,id1217141,5.486946
3,id2150126,8.378023
4,id1598245,5.018014
5,id0668992,7.829856
6,id1765014,7.427616
7,id0898117,7.065222
8,id3905224,11.710353
9,id1543102,6.068488


In [78]:
#.csv로 저장
final.to_csv("final.csv", index=False)